In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random

In [2]:
year = 2013
url = "https://www.melon.com/chart/age/list.htm?idx=1&chartType=YE&chartGenre=KPOP&chartDate={}&moved=Y".format(year)
headers = {
    "User-Agent" : UserAgent().chrome
}

In [3]:
response = requests.get(url, headers=headers)
response

<Response [200]>

In [4]:
dom = BeautifulSoup(response.text, 'html.parser')
elements = dom.select("#frm > table > tbody tr")
len(elements)

100

In [5]:
element = elements[0]

In [6]:
data = {
    "title" : element.select_one('a > .odd_span').text.split('상')[0].strip(),
    "artist" : element.select_one('div > .fc_mgray').text,
    "link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
}

In [7]:
datas = []
for element in elements:
    data = {
        "title" : element.select_one('a > .odd_span').text.split('상')[0].strip(),
        "artist" : element.select_one('div > .fc_mgray').text,
        "link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
    }
    datas.append(data)

In [8]:
import pandas as pd

In [9]:
song_df = pd.DataFrame(datas)
song_df.tail()

,artist,link,title
95,버벌진트,https://www.melon.com/song/detail.htm?songId=4...,비범벅 (Feat. 범키)
96,2NE1,https://www.melon.com/song/detail.htm?songId=4...,그리워해요
97,빈지노 (Beenzino),https://www.melon.com/song/detail.htm?songId=3...,Boogie On & On
98,버벌진트,https://www.melon.com/song/detail.htm?songId=4...,시작이 좋아 (Feat. 강민희 Of 미스에스)
99,써니힐,https://www.melon.com/song/detail.htm?songId=3...,Goodbye To Romance


In [10]:
link = song_df.loc[0, 'link']
print(link)

https://www.melon.com/song/detail.htm?songId=3726617


In [33]:
headers = {
    "User-Agent" : UserAgent().chrome
}
response = requests.get(link, headers=headers)
dom = BeautifulSoup(response.content, 'html.parser')
response

<Response [406]>

In [12]:
elements = dom.select_one('div.entry > div.meta > dl > dd:nth-child(6)').text
elements

'랩/힙합'

In [13]:
def get_genre(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    
    return dom.select_one('div.entry > div.meta > dl > dd:nth-child(6)').text

In [14]:
def song_details(year):

        url = "https://www.melon.com/chart/age/list.htm?idx=1&chartType=YE&chartGenre=KPOP&chartDate={}&moved=Y".format(year)
        headers = {
            "User-Agent" : UserAgent().chrome
        }
        response = requests.get(url, headers=headers)
        dom = BeautifulSoup(response.text, 'html.parser')
        elements = dom.select("#frm > table > tbody tr")
        
        datas = []
        for element in elements:
            data = {
                "Title" : element.select_one('a > .odd_span').text.split('상')[0].strip(),
                "Artist" : element.select_one('div > .fc_mgray').text,
                "Year" : year,
                "Link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
            }
            datas.append(data)
    
        return datas

### 1964~1975

In [ ]:
# 장르 수집 함수
# 한꺼번에 너무 많이 수집하면 도중에 response 406이 발생하여 상세 페이지 접속이 불가함
# 년도를 나눠서 수집하면 정상적으로 수집할 수 있으나 한 번 실행하고 나면 response 406이 발생하기 때문에 한시간 정도 쉬어주면서 다시 실행해야 함

In [16]:
year = list(range(1964, 1976))

In [17]:
result = []
for y in year:
    result.extend(song_details(y))

In [18]:
first_df = pd.DataFrame(result)

In [19]:
first_df['Genre'] = first_df['Link'].apply(get_genre)
first_df.tail()

,Artist,Link,Title,Year,Genre
390,김정호,https://www.melon.com/song/detail.htm?songId=4...,푸른 하늘 아래로,1975,포크/블루스
391,김정호,https://www.melon.com/song/detail.htm?songId=1...,하얀 나비 (Narr. 이윤선),1975,성인가요
392,송대관,https://www.melon.com/song/detail.htm?songId=5...,해뜰 날,1975,성인가요
393,김세환,https://www.melon.com/song/detail.htm?songId=3...,화가 났을까,1975,포크/블루스
394,정미조,https://www.melon.com/song/detail.htm?songId=1...,휘파람을 부세요,1975,발라드


In [20]:
first_df.to_csv('./datas/first.csv')

### 1976~1985

In [33]:
year = list(range(1976, 1986))

In [34]:
result = []
for y in year:
    result.extend(song_details(y))

In [35]:
second_df = pd.DataFrame(result)
second_df.tail()

,Artist,Link,Title,Year
478,김연자,https://www.melon.com/song/detail.htm?songId=1...,영동부르스,1985
479,김학래,https://www.melon.com/song/detail.htm?songId=2...,사랑과 약속,1985
480,황선형,https://www.melon.com/song/detail.htm?songId=1...,물망초,1985
481,민지,https://www.melon.com/song/detail.htm?songId=9...,당신이 주는 것,1985
482,이리,https://www.melon.com/song/detail.htm?songId=9...,어제,1985


In [63]:
second_df['Genre'] = second_df['Link'].apply(get_genre)
second_df.tail()

,Artist,Link,Title,Year,Genre
478,김연자,https://www.melon.com/song/detail.htm?songId=1...,영동부르스,1985,"발라드, 성인가요"
479,김학래,https://www.melon.com/song/detail.htm?songId=2...,사랑과 약속,1985,발라드
480,황선형,https://www.melon.com/song/detail.htm?songId=1...,물망초,1985,성인가요
481,민지,https://www.melon.com/song/detail.htm?songId=9...,당신이 주는 것,1985,성인가요
482,이리,https://www.melon.com/song/detail.htm?songId=9...,어제,1985,성인가요


In [64]:
second_df.to_csv('./datas/second.csv')

### 1986~1995

In [65]:
year = list(range(1986, 1996))

In [66]:
result = []
for y in year:
    result.extend(song_details(y))

In [67]:
third_df = pd.DataFrame(result)
third_df.tail()

,Artist,Link,Title,Year
991,EOS(이오에스),https://www.melon.com/song/detail.htm?songId=6...,넌 남이 아냐 (House Ver.),1995
992,신승훈,https://www.melon.com/song/detail.htm?songId=5...,오랜 이별뒤에,1995
993,김범수,https://www.melon.com/song/detail.htm?songId=6...,Happy End (Edit Ver.),1995
994,김원준,https://www.melon.com/song/detail.htm?songId=6...,넌 내꺼,1995
995,최연제,https://www.melon.com/song/detail.htm?songId=6...,너를 잊을 수 없어,1995


In [96]:
third_df['Genre'] = third_df['Link'].apply(get_genre)
third_df.tail()

,Artist,Link,Title,Year,Genre
991,EOS(이오에스),https://www.melon.com/song/detail.htm?songId=6...,넌 남이 아냐 (House Ver.),1995,댄스
992,신승훈,https://www.melon.com/song/detail.htm?songId=5...,오랜 이별뒤에,1995,발라드
993,김범수,https://www.melon.com/song/detail.htm?songId=6...,Happy End (Edit Ver.),1995,"재즈, 애시드/퓨전/팝"
994,김원준,https://www.melon.com/song/detail.htm?songId=6...,넌 내꺼,1995,댄스
995,최연제,https://www.melon.com/song/detail.htm?songId=6...,너를 잊을 수 없어,1995,발라드


In [97]:
third_df.to_csv('./datas/third.csv')

### 2001~2005

In [15]:
year = list(range(2001, 2006))

In [16]:
result = []
for y in year:
    result.extend(song_details(y))

In [17]:
fourth_df = pd.DataFrame(result)
fourth_df.tail()

,Artist,Link,Title,Year
494,거북이,https://www.melon.com/song/detail.htm?songId=6...,빙고 (아싸),2005
495,전혜빈,https://www.melon.com/song/detail.htm?songId=9...,2 A.M (2:00am),2005
496,슈가,https://www.melon.com/song/detail.htm?songId=9...,현명한 이별,2005
497,나윤권,https://www.melon.com/song/detail.htm?songId=9...,기대,2005
498,클래지콰이,https://www.melon.com/song/detail.htm?songId=8...,She Is,2005


In [18]:
fourth_df['Genre'] = fourth_df['Link'].apply(get_genre)
fourth_df.tail()

,Artist,Link,Title,Year,Genre
494,거북이,https://www.melon.com/song/detail.htm?songId=6...,빙고 (아싸),2005,댄스
495,전혜빈,https://www.melon.com/song/detail.htm?songId=9...,2 A.M (2:00am),2005,댄스
496,슈가,https://www.melon.com/song/detail.htm?songId=9...,현명한 이별,2005,댄스
497,나윤권,https://www.melon.com/song/detail.htm?songId=9...,기대,2005,발라드
498,클래지콰이,https://www.melon.com/song/detail.htm?songId=8...,She Is,2005,"일렉트로니카, 국내드라마"


In [19]:
fourth_df.to_csv('./datas/fifth.csv')

### 2011~2015

In [28]:
year = list(range(2011, 2016))

In [29]:
result = []
for y in year:
    result.extend(song_details(y))

In [30]:
fifth_df = pd.DataFrame(result)
fifth_df.tail()

,Artist,Link,Title,Year
495,블락비 바스타즈,https://www.melon.com/song/detail.htm?songId=5...,품행제로,2015
496,김범수,https://www.melon.com/song/detail.htm?songId=5...,사랑의 시작은 고백에서부터,2015
497,가인,https://www.melon.com/song/detail.htm?songId=5...,Apple (Feat. 박재범),2015
498,지코 (ZICO),https://www.melon.com/song/detail.htm?songId=5...,아프다,2015
499,소유 (SOYOU),https://www.melon.com/song/detail.htm?songId=7...,모르나봐,2015


In [31]:
fifth_df['Genre'] = fifth_df['Link'].apply(get_genre)
fifth_df.tail()

,Artist,Link,Title,Year,Genre
495,블락비 바스타즈,https://www.melon.com/song/detail.htm?songId=5...,품행제로,2015,댄스
496,김범수,https://www.melon.com/song/detail.htm?songId=5...,사랑의 시작은 고백에서부터,2015,"발라드, 국내드라마"
497,가인,https://www.melon.com/song/detail.htm?songId=5...,Apple (Feat. 박재범),2015,댄스
498,지코 (ZICO),https://www.melon.com/song/detail.htm?songId=5...,아프다,2015,"발라드, 랩/힙합, 국내드라마"
499,소유 (SOYOU),https://www.melon.com/song/detail.htm?songId=7...,모르나봐,2015,"발라드, 국내드라마"


In [32]:
fifth_df.to_csv('./datas/seventh.csv')